In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
# loading the dataset
%time data = pd.read_excel("all_matches.xlsx")

Wall time: 1min 54s


In [3]:
df = data

In [4]:
# choosing necessary columns
frame = df.drop(
    columns=[
        "winner_age",
        "loser_age",
        "tourney_id",
        "tourney_name",
        "draw_size",
        "tourney_level",
        "match_num",
        "winner_id",
        "winner_seed",
        "winner_entry",
        "winner_ioc",
        "loser_id",
        "loser_seed",
        "loser_entry",
        "loser_ioc",
        "score",
        "best_of",
        "round",
        "minutes",
        "winner_rank",
        "loser_rank",
        "winner_rank_points",
        "loser_rank_points",
        "w_SvGms",
        "l_SvGms",
        "w_1stIn",
        "l_1stIn",
    ]
)

In [5]:
frame["w_serves_won"] = 0
frame["l_serves_won"] = 0
frame["w_serves_won"] = frame["w_1stWon"] + frame["w_2ndWon"]
frame["l_serves_won"] = frame["l_1stWon"] + frame["l_2ndWon"]

frame = frame.drop(
    columns=[
        "w_1stWon",
        "w_2ndWon",
        "l_1stWon",
        "l_2ndWon",
    ]
)

In [6]:
# replacing missing height with an average height
frame["winner_ht"] = frame["winner_ht"].fillna(value=round(frame["winner_ht"].mean()))
frame["loser_ht"] = frame["loser_ht"].fillna(value=round(frame["loser_ht"].mean()))

# replacing missing hand with the most common value
frame["winner_hand"] = frame["winner_hand"].fillna(value=frame["winner_hand"].mode()[0])
frame["loser_hand"] = frame["loser_hand"].fillna(value=frame["loser_hand"].mode()[0])

# replacing missing surface with the most common value
frame["surface"] = frame["surface"].fillna(value=frame["surface"].mode()[0])

for i in frame.columns[10:]:
    frame[i] = frame[i].fillna(0)

In [7]:
pd.options.mode.chained_assignment = None

# renaming columns
frame.columns = np.array(['surface', 'date', 'p1', 'p1_hand', 'p1_ht', 'p2', 'p2_hand', 'p2_ht',
                          'p1_ace', 'p1_df', 'p1_serves_made','p1_bp_saved', 'p1_bp_faced',
                          'p2_ace', 'p2_df', 'p2_serves_made','p2_bp_saved', 'p2_bp_faced',
                          'p1_serves_won', 'p2_serves_won'])

frame['winner'] = frame['p1']
frame.drop(index=frame.loc[frame['p2'].isnull()].index, inplace=True)

frame = frame[['surface', 'date', 'p1', 'p2', 'winner', 'p1_hand', 'p1_ht', 'p1_ace', 'p1_df',
               'p1_serves_won','p1_serves_made', 'p1_bp_saved', 'p1_bp_faced', 'p2_hand', 
               'p2_ht', 'p2_ace', 'p2_df', 'p2_serves_won', 'p2_serves_made','p2_bp_saved', 'p2_bp_faced']]

pd.options.mode.chained_assignment = None
l1 = np.array(['p1', 'p1_hand', 'p1_ht', 'p1_ace', 'p1_df', 'p1_serves_won',
               'p1_serves_made', 'p1_bp_saved', 'p1_bp_faced'])
l2 = np.array(['p2', 'p2_hand', 'p2_ht', 'p2_ace', 'p2_df', 'p2_serves_won',
               'p2_serves_made', 'p2_bp_saved', 'p2_bp_faced'])


In [8]:
# mixing players
def mixing_players():
    for i in frame.index:
        rand = np.random.randint(2)
        if rand == True:
            for j, k in zip(l1, l2):
                frame[j][i], frame[k][i] = frame[k][i], frame[j][i]
                
%time mixing_players()

Wall time: 20min 47s


In [9]:
unique_player = np.unique(frame[["p1", "p2"]].values)
unique_surface = np.unique(frame["surface"].values)


def init_df_data():
    player_index = ["win", "total", "last5", "left_win", "left_total", "right_win", 
                    "right_total", "uni_win", "uni_total", "serves_won", "serves_total",
                    "bp_saved","bp_faced"]
    player = pd.DataFrame(0, index=player_index, columns=unique_player)
    player.loc["elo"] = 1500
    player.loc["height"] = 0
    player.loc["hand"] = 0

    surface_index = pd.MultiIndex.from_product([unique_surface, ["win", "total"]])
    surface = pd.DataFrame(0, index=surface_index, columns=unique_player)    
    
    versus_index = pd.MultiIndex.from_product([unique_player, ["won", "lost"]])
    versus = pd.DataFrame(0.0, index=versus_index, columns=unique_player)

    return player, surface, versus


frame["p1_break"] = 0.0
frame["p2_break"] = 0.0
frame["p1_serve"] = 0.0
frame["p2_serve"] = 0.0
frame["p1_elo"] = 0.0
frame["p2_elo"] = 0.0
frame["p1_hand_form"] = 0.0
frame["p2_hand_form"] = 0.0
frame["p1_win_h2h"] = 0.0
frame["p2_win_h2h"] = 0.0
frame["p1_win_last5"] = 0.0
frame["p2_win_last5"] = 0.0
frame["p1_win_overall"] = 0.0
frame["p2_win_overall"] = 0.0
frame["p1_surface_overall"] = 0.0
frame["p2_surface_overall"] = 0.0

In [10]:
def elo(elo1, elo2, n1, n2, result):
    def k(n):
        return 250 / (pow((5 + n), 0.4))

    def pi(first, second):
        return pow(1 + pow(10, (second - first) / 400), -1)

    e1 = elo1 + k(n1) * (result - pi(elo1, elo2))
    e2 = elo2 + k(n2) * ((1 - result) - pi(elo2, elo1))

    return e1, e2

In [11]:
# replacing winner name with 1 or 0
%time frame['winner'] = np.where(frame["p1"].values == frame["winner"].values, 1,0 )

Wall time: 11 ms


In [12]:
# adding interesting features like win_overall, surface_overall etc.
def fill_params(index):
    p1_name = frame["p1"][index]
    p2_name = frame["p2"][index]
    surface = frame["surface"][index]
    p1_hand = frame["p1_hand"][index]
    p2_hand = frame["p2_hand"][index]
    
    df_player[p1_name]['height'] = frame['p1_ht'][index]
    df_player[p2_name]['height'] = frame['p2_ht'][index]
        
    if df_player[p1_name]['hand'] == 0:
        if p1_hand == 'L':
            df_player[p1_name]['hand'] = 1
        elif p1_hand == 'R':
            df_player[p1_name]['hand'] = 2
        elif p1_hand == 'U':
            df_player[p1_name]['hand'] = 3
            
    if df_player[p2_name]['hand'] == 0:
        if p2_hand == 'L':
            df_player[p2_name]['hand'] = 1
        elif p2_hand == 'R':
            df_player[p2_name]['hand'] = 2
        elif p2_hand == 'U':
            df_player[p2_name]['hand'] = 3

    if df_player[p1_name]["total"] > 0:
        frame["p1_win_overall"][index] = (df_player[p1_name]["win"] / df_player[p1_name]["total"])
    
    if df_player[p2_name]["total"] > 0:
        frame["p2_win_overall"][index] = (df_player[p2_name]["win"] / df_player[p2_name]["total"])
        
    if df_player[p1_name]["serves_total"] > 0:
        frame["p1_serve"][index] = df_player[p1_name]["serves_won"]/df_player[p1_name]["serves_total"]
        
    if df_player[p2_name]["serves_total"] > 0:
        frame["p2_serve"][index] = df_player[p2_name]["serves_won"]/df_player[p2_name]["serves_total"]
        
    if df_player[p1_name]["bp_faced"] > 0:
        frame["p1_break"][index] = df_player[p1_name]["bp_saved"]/df_player[p1_name]["bp_faced"]
        
    if df_player[p2_name]["bp_faced"] > 0:
        frame["p2_break"][index] = df_player[p2_name]["bp_saved"]/df_player[p2_name]["bp_faced"]
    
    frame['p1_win_h2h'][index] = df_versus[p1_name][p2_name]['won']
    frame['p2_win_h2h'][index] = df_versus[p2_name][p1_name]['won']
    
    frame["p1_win_last5"][index] = df_player[p1_name]["last5"]
    frame["p2_win_last5"][index] = df_player[p2_name]["last5"]

    if df_surface[p1_name][surface]["total"] > 0:
        frame["p1_surface_overall"][index] = (df_surface[p1_name][surface]["win"] / df_surface[p1_name][surface]["total"])
        
    if df_surface[p2_name][surface]["total"] > 0:
        frame["p2_surface_overall"][index] = (df_surface[p2_name][surface]["win"] / df_surface[p2_name][surface]["total"])
        

    frame["p1_elo"][index] = df_player[p1_name]["elo"]
    frame["p2_elo"][index] = df_player[p2_name]["elo"]

    if p2_hand == "R" and df_player[p1_name]["right_total"] > 0:
        frame["p1_hand_form"][index] = (
            df_player[p1_name]["right_win"] / df_player[p1_name]["right_total"]
        )
    elif p2_hand == "L" and df_player[p1_name]["left_total"] > 0:
        frame["p1_hand_form"][index] = (
            df_player[p1_name]["left_win"] / df_player[p1_name]["left_total"]
        )
    elif p2_hand == "U" and df_player[p1_name]["uni_total"] > 0:
        frame["p1_hand_form"][index] = (
            df_player[p1_name]["uni_win"] / df_player[p1_name]["uni_total"]
        )

    if p1_hand == "R" and df_player[p2_name]["right_total"] > 0:
        frame["p2_hand_form"][index] = (
            df_player[p2_name]["right_win"] / df_player[p2_name]["right_total"]
        )
    elif p1_hand == "L" and df_player[p2_name]["left_total"] > 0:
        frame["p2_hand_form"][index] = (
            df_player[p2_name]["left_win"] / df_player[p2_name]["left_total"]
        )
    elif p1_hand == "U" and df_player[p2_name]["uni_total"] > 0:
        frame["p2_hand_form"][index] = (
            df_player[p2_name]["uni_win"] / df_player[p2_name]["uni_total"]
        )

def calc_params(index):
    p1_name = frame["p1"][index]
    p2_name = frame["p2"][index]
    surface = frame["surface"][index]
    p1_hand = frame["p1_hand"][index]
    p2_hand = frame["p2_hand"][index]
    winner = frame["winner"][index]
    

    df_player[p1_name]["elo"], df_player[p2_name]["elo"] = elo(
        df_player[p1_name]["elo"],
        df_player[p2_name]["elo"],
        df_player[p1_name]["total"],
        df_player[p2_name]["total"],
        winner,
    )

    df_player[p1_name]["total"] += 1
    df_player[p2_name]["total"] += 1

    df_surface[p1_name][surface]["total"] += 1
    df_surface[p2_name][surface]["total"] += 1

    df_player[p1_name]["serves_won"] += frame["p1_serves_won"][index]
    df_player[p2_name]["serves_won"] += frame["p2_serves_won"][index]

    df_player[p1_name]["serves_total"] += frame["p1_serves_made"][index]
    df_player[p2_name]["serves_total"] += frame["p2_serves_made"][index]

    df_player[p1_name]["bp_saved"] += frame["p1_bp_saved"][index]
    df_player[p2_name]["bp_saved"] += frame["p2_bp_saved"][index]

    df_player[p1_name]["bp_faced"] += frame["p1_bp_faced"][index]
    df_player[p2_name]["bp_faced"] += frame["p2_bp_faced"][index]
    
    
    if p2_hand == "R":
        df_player[p1_name]["right_total"] += 1
    elif p2_hand == "L":
        df_player[p1_name]["left_total"] += 1
    else:
        df_player[p1_name]["uni_total"] += 1

    if p1_hand == "R":
        df_player[p2_name]["right_total"] += 1
    elif p1_hand == "L":
        df_player[p2_name]["left_total"] += 1
    else:
        df_player[p2_name]["uni_total"] += 1

    if winner:
        df_player[p1_name]["win"] += 1
        df_surface[p1_name][surface]["win"] += 1
        df_versus[p1_name][p2_name]["won"] += 1
        df_versus[p2_name][p1_name]["lost"] += 1

        if p2_hand == "R":
            df_player[p1_name]["right_win"] += 1
        elif p2_hand == "L":
            df_player[p1_name]["left_win"] += 1
        else:
            df_player[p1_name]["uni_win"] += 1

        if df_player[p1_name]["last5"] < 5:
            df_player[p1_name]["last5"] += 1
        if df_player[p2_name]["last5"] > 0:
            df_player[p2_name]["last5"] -= 1

    else:
        df_player[p2_name]["win"] += 1
        df_surface[p2_name][surface]["win"] += 1
        df_versus[p2_name][p1_name]["won"] += 1
        df_versus[p1_name][p2_name]["lost"] += 1

        if p1_hand == "R":
            df_player[p2_name]["right_win"] += 1
        elif p1_hand == "L":
            df_player[p2_name]["left_win"] += 1
        else:
            df_player[p2_name]["uni_win"] += 1

        if df_player[p1_name]["last5"] > 0:
            df_player[p1_name]["last5"] -= 1
        if df_player[p2_name]["last5"] < 5:
            df_player[p2_name]["last5"] += 1

In [13]:
df_player, df_surface, df_versus = init_df_data()

def computation():
    for index in frame.index:
        fill_params(index)
        calc_params(index)

In [14]:
%time computation()
frame.fillna(0, inplace=True)

Wall time: 38min 13s


In [15]:
# creating frame 'result' with final output
result = pd.DataFrame(0.0, index=frame.index,
    columns=[
        "tourney_name",
        "round",
        "date",
        "p1",
        "p2",
        "winner",
        "diff_ht",
        "diff_bp",
        "diff_sp",
        "diff_elo",
        "diff_hand",
        "diff_win_h2h",
        "diff_win_last5",
        "diff_win_overall",
        "diff_surface_overall",
    ]
)
result['tourney_name'] = df['tourney_name']
result['round'] = df['round']
result["date"] =  frame["date"]
result["p1"] = frame["p1"]
result["p2"] = frame["p2"]
result["winner"] = frame["winner"]
result["diff_ht"] = frame["p1_ht"] - frame["p2_ht"]
result["diff_bp"] = frame["p1_break"] - frame["p2_break"]
result["diff_sp"] = frame["p1_serve"] - frame["p2_serve"]
result["diff_elo"] = frame["p1_elo"] - frame["p2_elo"]
result["diff_hand"] = frame["p1_hand_form"] - frame["p2_hand_form"]
result["diff_win_h2h"] = frame["p1_win_h2h"] - frame["p2_win_h2h"]
result["diff_win_last5"] = frame["p1_win_last5"] - frame["p2_win_last5"]
result["diff_win_overall"] = frame["p1_win_overall"] - frame["p2_win_overall"]
result["diff_surface_overall"] = frame["p1_surface_overall"] - frame["p2_surface_overall"]
    
result.fillna(0, inplace=True)

In [16]:
#saving necessary frames
%time result.to_csv('result.csv')
%time df_player.to_csv('df_player.csv')
%time df_surface.to_csv('df_surface.csv')
%time df_versus.to_csv('df_versus.csv')

Wall time: 3.04 s
Wall time: 58 ms
Wall time: 49 ms
